In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pickle
from types import SimpleNamespace
from utils import bench
import itertools

## Load data

In [ ]:
import glob
files = sorted(glob.glob('./bench_data/2018-05-11T1036/RIO*pickle'))
files += sorted(glob.glob('./bench_data/2018-05-11T1152/RIO*pickle'))


dd_all = [pickle.load(open(file, 'rb')) for file in files]

In [ ]:
dd = [sorted(v, key=lambda d: d.t_total)[0] 
      for _, v in itertools.groupby(dd_all, lambda d: d.params.nthreads)]

sts = [bench.unpack_stats(d, ms=True) for d in dd]
nthreads = np.array([d.params.nthreads for d in dd])
dd_by_thread = {d.params.nthreads:d for d in dd}
nth_to_idx = {d.params.nthreads:i for i,d in enumerate(dd) }

figs = {}

#[(d.params.nthreads, d.t_total) for d in dd]

## Scaling with more threads

In [ ]:
fig = plt.figure(figsize=(12,8))
figs['threads'] = fig

best_idx = bench.plot_stats_results(dd, fig=fig)

## In depth stats for single threaded case

In [ ]:
print(bench.gen_stats_report(dd[0]))
fig = plt.figure(figsize=(12,8))
figs['single-thread-in-depth'] = fig

bench.plot_results(dd[0].stats, fig=fig, #cc=('C0', 'C9', 'C4', 'C7')
);


In [ ]:
reports = (bench.gen_stats_report(dd[0], 'One Thread'),
           bench.gen_stats_report(dd[best_idx], 'Lowest overall latency'))
print(bench.join_reports(*reports))



fig = plt.figure(figsize=(12,6))
figs['comparison'] = fig

bench.plot_comparison(fig, [sts[0], sts[best_idx]],
                      nochunk=True,
                      threshs=[200, 100, 60],
                      alpha=0.4,
                      names=['c1', 'c{}'.format(dd[best_idx].params.nthreads)]
                     )

## Sample image

In [ ]:
pix = np.load('./bench_data/tiles.npz')['data']
fig = plt.figure(figsize=(4,4))
figs['sample-image'] = fig
ax = fig.add_axes((0,0,1,1))
ax.imshow(pix[0], vmin=0, vmax=255, cmap='jet')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

## Latency hiding graph

In [ ]:
def add_hbar_plot(ax, st, n=None, height=1, **kwargs):
    if n is None:
        n = st.t0.shape[0]
    
    for i in range(n):
        y = i + 1
        width = st.t_total[i]
        t0 = st.t0[i]
        ax.barh(y, left=t0, width=width, height=height, **kwargs)



fig = plt.figure(figsize=(12,6))
figs['latency-hiding'] = fig
axs = [fig.add_subplot(121), fig.add_subplot(122)]
ii = (0,1, 2, best_idx)
for i, c in zip(ii, ['C0', 'C1', 'C3', 'C2']):
    st = sts[i]
    for ax in axs:
        add_hbar_plot(ax, st, n=40, 
                      color=c, 
                      alpha=0.4, 
                      linewidth=0, 
                      label='c{}'.format(dd[i].params.nthreads))
        ax.set_xlabel('ms')
        ax.axis( ax.axis()[:2] + (1,41))


axs[1].axis((0,750, *ax.axis()[2:]))
axs[1].yaxis.set_visible(False)
axs[1].legend()
fig.tight_layout()


In [ ]:
fig = plt.figure(figsize=(16,6))
figs['fps'] = fig
ax = fig.add_subplot(111)

mf = np.r_[[np.median(st.fps) for st in sts]]

for st in sts:
    ax.plot(st.fps, 'k-', alpha=0.4, linewidth=0.7)

for n in [1, 8, 16, 24, nthreads[best_idx], nthreads.max()]:
    idx = nth_to_idx[n]
    st = sts[idx]
    ax.plot(st.fps, '-', linewidth=2, label='c{}'.format(st.nthreads))



ax.set_xlabel('Files proccessed')
ax.set_ylabel('Files per second')
ax.axis((-3, st.t_end.shape[0]+10) + ax.axis()[2:])
ax.legend(loc='upper left');
fig.tight_layout()



In [ ]:
warmup_time = np.array([np.median([st.t_open for st in d._warmup.stats]) for d in dd])

fig = plt.figure(figsize=(4,4))
figs['warmup'] = fig
ax = fig.add_subplot(111)
ax.barh(nthreads, warmup_time*1000, height=0.5, alpha=0.7)
ax.axis([0,2100, 0.5, nthreads[-1]+1.5])
ax.yaxis.set_ticks([1, 8, 16, 24, 32, nthreads[-1]])
ax.set_xlabel('Median time to open first file (ms)')
ax.set_ylabel('Number of threads')
fig.tight_layout()

In [ ]:
!mkdir -p report_images

In [ ]:
defaults = dict(dpi=100)
overrides = dict(warmup=dict(dpi=200))


for name, fig in figs.items():
    for fmt in ['svg', 'png']:
        fname = './report_images/{name}.{fmt}'.format(name=name, fmt=fmt)
        print('Saving to: ' + fname)
        fig.savefig(fname, **overrides.get(name, defaults))